In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression 
import matplotlib.pylab as pylab
import math,time
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout, Activation,Flatten
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from keras.layers import Bidirectional
test_set_size_percentage = 20 
import pandas as pd
from pandas import read_csv
from sklearn.metrics import r2_score,mean_squared_error 

Using TensorFlow backend.


In [2]:
data=pd.read_csv('dataset/iron_oilmonthly.csv',parse_dates=['date'])
data=data.dropna()
#df=data[['oildata','date']]
data=data.set_index('date')
data=data.dropna()
data.head()

,ironprice,oildata
date,,
2010-10-31,148.48,79.662
2010-11-30,160.55,79.662
2010-12-31,168.53,79.662
2011-01-31,179.63,79.662
2011-02-28,187.18,79.662


In [3]:
data['oildata_lag']=data['oildata'].shift(1)
#data=data.set_index('date')
data['rolling_mean_price']=data['oildata_lag'].rolling(2, min_periods=1).sum()
data=data.dropna()

In [4]:
# function for min-max normalization of Iron Ore 
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
def normalize_data(df):
    df['oildata'] = min_max_scaler.fit_transform(df.oildata.values.reshape(-1,1))
    df['oildata_lag'] = min_max_scaler.fit_transform(df.oildata_lag.values.reshape(-1,1))
    df['ironprice'] = min_max_scaler.fit_transform(df.ironprice.values.reshape(-1,1))
    df['rolling_mean_price'] = min_max_scaler.fit_transform(df.rolling_mean_price.values.reshape(-1,1))
    return df

In [5]:
# normalize stock
df = data.copy()
df = normalize_data(df)
df.head()

,ironprice,oildata,oildata_lag,rolling_mean_price
date,,,,
2010-11-30,0.817977,0.551036,0.551036,0.131441
2010-12-31,0.872522,0.551036,0.551036,0.559881
2011-01-31,0.948394,0.551036,0.551036,0.559881
2011-02-28,1.000000,0.551036,0.551036,0.559881
2011-03-31,0.878195,0.551036,0.551036,0.559881


In [6]:
train,test=df[:80],df[80:]

In [7]:
train.tail()

,ironprice,oildata,oildata_lag,rolling_mean_price
date,,,,
2017-02-28,0.328571,0.172056,0.167704,0.179077
2017-03-31,0.316610,0.169818,0.172056,0.164364
2017-04-30,0.201777,0.221458,0.169818,0.165462
2017-05-31,0.141832,0.173263,0.221458,0.191093
2017-06-30,0.116063,0.260908,0.173263,0.192881


In [8]:
test.head()

,ironprice,oildata,oildata_lag,rolling_mean_price
date,,,,
2017-07-31,0.176760,0.268411,0.260908,0.213348
2017-08-31,0.230417,0.278490,0.268411,0.262715
2017-09-30,0.193848,0.246857,0.278490,0.271837
2017-10-31,0.137867,0.257753,0.246857,0.260654
2017-11-30,0.153999,0.235301,0.257753,0.249895


In [9]:
print(len(train), len(test))

80 23


In [10]:
train_data_raw = train.as_matrix()
test_data_raw = test.as_matrix()

C:\Users\sumon\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\sumon\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [11]:
print("train data shape = ", train_data_raw.shape)
print("test data shape = ",test_data_raw.shape)

train data shape =  (80, 4)
test data shape =  (23, 4)


In [12]:
x_train = train_data_raw[:,1:] #without Iron price column of train
y_train = train_data_raw[:,0] #Only Iron price column of train

x_test = test_data_raw[:,1:] #without Iron price column of test
y_test = test_data_raw[:,0]  #Only Iron price column of test

In [13]:
x_train[0:5]

array([[0.55103581, 0.55103581, 0.13144078],
       [0.55103581, 0.55103581, 0.55988062],
       [0.55103581, 0.55103581, 0.55988062],
       [0.55103581, 0.55103581, 0.55988062],
       [0.55103581, 0.55103581, 0.55988062]])

In [14]:
y_train[0:5]

array([0.81797676, 0.87252221, 0.94839371, 1.        , 0.87819549])

In [15]:
train_data_raw[0:5]

array([[0.81797676, 0.55103581, 0.55103581, 0.13144078],
       [0.87252221, 0.55103581, 0.55103581, 0.55988062],
       [0.94839371, 0.55103581, 0.55103581, 0.55988062],
       [1.        , 0.55103581, 0.55103581, 0.55988062],
       [0.87819549, 0.55103581, 0.55103581, 0.55988062]])

In [16]:
# def denormalize(df, normalized_value): 
#     df = df['Price'].values.reshape(-1,1)
#     print(df[:5])
#     normalized_value = normalized_value.reshape(-1,1)
#     #return df.shape, p.shape
#     print(normalized_value[0:5])
#     min_max_scaler = sklearn.preprocessing.MinMaxScaler()
#     a = min_max_scaler.fit_transform(df)
#     print(a[:5])
#     new = min_max_scaler.inverse_transform(normalized_value)
#     print(new[:10])
#     return new

# #newp = denormalize(test_value_store, pred_price)
# newy_test = denormalize(data, y_test)

In [17]:
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (80, 3)
y_train.shape =  (80,)
x_test.shape =  (23, 3)
y_test.shape =  (23,)


In [18]:
#Reshape data for (Sample,Timestep,Features) 
x_train = x_train.reshape((x_train.shape[0],1,x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0],1,x_test.shape[1]))
#x_forecast = x_forecast.reshape((x_forecast.shape[0],1,x_test.shape[1]))
x_test.shape

(23, 1, 3)

In [19]:
(train_dim, train_timestep , train_feat) = x_train.shape
(test_dim, test_timestep , test_feat) = x_test.shape

In [20]:
print("Build Model")
 
model = Sequential()

model.add(LSTM(32, input_shape=(train_timestep,train_feat), return_sequences=True))
#model.add(Dropout(0.3))

model.add(LSTM(16, input_shape=(train_timestep,train_feat), return_sequences=False))
#model.add(Dropout(0.3))

model.add(Dense(8,kernel_initializer="uniform",activation='relu'))        
model.add(Dense(1,kernel_initializer="uniform",activation='linear'))

# adam = keras.optimizers.Adam(decay=0.2)

start = time.time()
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
print("Compilation Time : ", time.time() - start)
"""
model = Sequential()

model.add(Bidirectional(LSTM(32, input_shape=(train_timestep,train_feat), return_sequences=True)))
#model.add(Dropout(0.3))

model.add(Dense(16,kernel_initializer="uniform",activation='relu'))  
model.add(Flatten())
model.add(Dense(1,kernel_initializer="uniform",activation='linear'))

# adam = keras.optimizers.Adam(decay=0.2)

start = time.time()
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
print("Compilation Time : ", time.time() - start)
"""

Build Model
Compilation Time :  0.031243562698364258


'\nmodel = Sequential()\n\nmodel.add(Bidirectional(LSTM(32, input_shape=(train_timestep,train_feat), return_sequences=True)))\n#model.add(Dropout(0.3))\n\nmodel.add(Dense(16,kernel_initializer="uniform",activation=\'relu\'))  \nmodel.add(Flatten())\nmodel.add(Dense(1,kernel_initializer="uniform",activation=\'linear\'))\n\n# adam = keras.optimizers.Adam(decay=0.2)\n\nstart = time.time()\nmodel.compile(loss=\'mse\',optimizer=\'adam\', metrics=[\'accuracy\'])\nprint("Compilation Time : ", time.time() - start)\n'

In [21]:
#Fit model with history to check for overfitting
history = model.fit(x_train,y_train,batch_size = 1,epochs=20,shuffle=True)


Epoch 1/20
80/80 [==============================] - 1s 8ms/step - loss: 0.2468 - accuracy: 0.0125
Epoch 2/20
80/80 [==============================] - 0s 2ms/step - loss: 0.1087 - accuracy: 0.0125
Epoch 3/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0651 - accuracy: 0.0125
Epoch 4/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0631 - accuracy: 0.0125
Epoch 5/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0622 - accuracy: 0.0125
Epoch 6/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0615 - accuracy: 0.0125
Epoch 7/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0612 - accuracy: 0.0125
Epoch 8/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0614 - accuracy: 0.0125
Epoch 9/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0598 - accuracy: 0.0125
Epoch 10/20
80/80 [==============================] - 0s 2ms/step - loss: 0.0605 - accuracy: 0.0125
Epoch 11/20
80/80 

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 32)             4608      
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 7,889
Trainable params: 7,889
Non-trainable params: 0
_________________________________________________________________


In [23]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [24]:
# load json and create model
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])

Loaded model from disk


In [25]:
pred_price = model.predict(x_test,batch_size=1,verbose=1)

23/23 [==============================] - 0s 6ms/step


In [26]:
pred_price

array([[0.27124348],
       [0.28602466],
       [0.2815585 ],
       [0.27534732],
       [0.2693615 ],
       [0.25225547],
       [0.24142827],
       [0.24995472],
       [0.2714091 ],
       [0.29315758],
       [0.31923988],
       [0.34285542],
       [0.36627212],
       [0.37416008],
       [0.3692234 ],
       [0.3790554 ],
       [0.40910336],
       [0.4289321 ],
       [0.4321242 ],
       [0.4288508 ],
       [0.4380776 ],
       [0.45654398],
       [0.43372604]], dtype=float32)

In [27]:
def denormalize(df, normalized_value): 
    df = df['ironprice'].values.reshape(-1,1)
    normalized_value = normalized_value.reshape(-1,1)
    #return df.shape, p.shape
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    a = min_max_scaler.fit_transform(df)
    new = min_max_scaler.inverse_transform(normalized_value)
    return new

newp = denormalize(data, pred_price)
newy_test = denormalize(data, y_test)

In [28]:
newp

array([[ 80.56292 ],
       [ 82.7254  ],
       [ 82.072014],
       [ 81.163315],
       [ 80.28759 ],
       [ 77.78497 ],
       [ 76.20096 ],
       [ 77.44838 ],
       [ 80.58715 ],
       [ 83.76896 ],
       [ 87.58479 ],
       [ 91.03975 ],
       [ 94.46561 ],
       [ 95.61962 ],
       [ 94.89738 ],
       [ 96.33581 ],
       [100.73182 ],
       [103.63277 ],
       [104.09977 ],
       [103.62087 ],
       [104.97075 ],
       [107.672386],
       [104.33412 ]], dtype=float32)

In [29]:
newy_test

array([[66.74],
       [74.59],
       [69.24],
       [61.05],
       [63.41],
       [71.74],
       [75.78],
       [77.65],
       [71.32],
       [66.33],
       [66.63],
       [66.86],
       [67.05],
       [68.02],
       [68.8 ],
       [72.02],
       [72.3 ],
       [68.9 ],
       [69.2 ],
       [76.2 ],
       [88.2 ],
       [86.5 ],
       [93.7 ]])

In [30]:
pred_oil_pirce = pd.concat([pd.DataFrame(newp),pd.DataFrame(newy_test)], axis=1) 
pred_oil_pirce.columns = ['Pred_Oil_Price','Actual_Oil_Price']
pred_oil_pirce['date'] = test.index
pred_oil_pirce = pred_oil_pirce.set_index('date')

In [31]:
pred_oil_pirce

,Pred_Oil_Price,Actual_Oil_Price
date,,
2017-07-31,80.562920,66.74
2017-08-31,82.725403,74.59
2017-09-30,82.072014,69.24
2017-10-31,81.163315,61.05
2017-11-30,80.287590,63.41
2017-12-31,77.784973,71.74
2018-01-31,76.200958,75.78
2018-02-28,77.448380,77.65
2018-03-31,80.587151,71.32


In [32]:
def model_score(model, x_train, y_train, x_test, y_test):
    trainScore = model.evaluate(x_train, y_train, verbose=0)
    print('Train Score: %.5f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

    testScore = model.evaluate(x_test, y_test, verbose=0)
    print('Test Score: %.5f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
    return trainScore[0], testScore[0]


model_score(model, x_train, y_train, x_test, y_test)

Train Score: 0.05734 MSE (0.24 RMSE)
Test Score: 0.02067 MSE (0.14 RMSE)


(0.05733979661017656, 0.020668458193540573)

In [33]:
def pred_accuracy(pred, actual):
    mape=np.mean(np.abs(pred-actual)/np.abs(actual))# MAPE
    me=np.mean(pred-actual)# ME
    mae=np.mean(np.abs(pred-actual))# MAE
    mpe=np.mean((pred-actual)/actual)# MPE
    return({'mape':mape,'me':me,'mae': mae,'mpe': mpe})
print(pred_accuracy(newp, newy_test))

{'mape': 0.2652203761944543, 'me': 18.66856962784477, 'mae': 18.68610184379246, 'mpe': 0.2649945910566863}


In [35]:
forecast_df = pd.read_csv('dataset/oil_forecast_data.csv')
forecast_df = forecast_df[['date','sarima_oilprice']]
forecast_df = forecast_df.set_index('date')
forecast_df

,sarima_oilprice
date,
2019-06-30,56.893623
2019-07-31,57.894870
2019-08-31,57.980675
2019-09-30,62.527335
2019-10-31,67.823088
2019-11-30,65.232596
2019-12-31,61.762357
2020-01-31,64.043057
2020-02-29,64.696372


In [36]:
#data['Price'].plot() 
forecast_df['oildata_lag']=forecast_df['sarima_oilprice'].shift(1)
#data=data.set_index('date')
forecast_df['rolling_mean_price']=forecast_df['oildata_lag'].rolling(2, min_periods=1).sum()
forecast_df=forecast_df.dropna()
forecast_df

,sarima_oilprice,oildata_lag,rolling_mean_price
date,,,
2019-07-31,57.894870,56.893623,56.893623
2019-08-31,57.980675,57.894870,114.788493
2019-09-30,62.527335,57.980675,115.875545
2019-10-31,67.823088,62.527335,120.508011
2019-11-30,65.232596,67.823088,130.350423
2019-12-31,61.762357,65.232596,133.055684
2020-01-31,64.043057,61.762357,126.994953
2020-02-29,64.696372,64.043057,125.805414
2020-03-31,71.018727,64.696372,128.739429


In [37]:
# function for min-max normalization of Iron Ore 
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
def normalize_data(df):
    df['sarima_oilprice'] = min_max_scaler.fit_transform(df.sarima_oilprice.values.reshape(-1,1))
    df['oildata_lag'] = min_max_scaler.fit_transform(df.oildata_lag.values.reshape(-1,1))
    df['rolling_mean_price'] = min_max_scaler.fit_transform(df.rolling_mean_price.values.reshape(-1,1))
    return df

In [38]:
# normalize stock
df = forecast_df.copy()
df = normalize_data(df)
df.head()

,sarima_oilprice,oildata_lag,rolling_mean_price
date,,,
2019-07-31,0.000000,0.000000,0.000000
2019-08-31,0.002992,0.033739,0.526198
2019-09-30,0.161551,0.036630,0.536078
2019-10-31,0.346232,0.189839,0.578182
2019-11-30,0.255893,0.368290,0.667638


In [39]:
forecast_data_raw = df.as_matrix()

C:\Users\sumon\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [40]:
forecast_data_raw.shape

(23, 3)

In [41]:
forecast_data_raw = forecast_data_raw.reshape((forecast_data_raw.shape[0],1,forecast_data_raw.shape[1]))
forecast_data_raw.shape

(23, 1, 3)

In [42]:
forecast_price = model.predict(forecast_data_raw,batch_size=1,verbose=1)

23/23 [==============================] - 0s 693us/step


In [43]:
forecast_price

array([[0.12016991],
       [0.21771084],
       [0.26185837],
       [0.3536803 ],
       [0.3853487 ],
       [0.33991137],
       [0.32488695],
       [0.34534213],
       [0.41241807],
       [0.50049865],
       [0.48182964],
       [0.3721444 ],
       [0.32992938],
       [0.35424286],
       [0.42641416],
       [0.52138096],
       [0.5327806 ],
       [0.47335994],
       [0.45867845],
       [0.48789248],
       [0.550209  ],
       [0.61921513],
       [0.620124  ]], dtype=float32)

In [44]:
def denormalize(df, normalized_value): 
    df = df['ironprice'].values.reshape(-1,1)
    normalized_value = normalized_value.reshape(-1,1)
    #return df.shape, p.shape
    min_max_scaler = sklearn.preprocessing.MinMaxScaler()
    a = min_max_scaler.fit_transform(df)
    new = min_max_scaler.inverse_transform(normalized_value)
    return new

newp = denormalize(data, forecast_price)
#newy_test = denormalize(data, y_test)

In [45]:
forecast_df.head()

,sarima_oilprice,oildata_lag,rolling_mean_price
date,,,
2019-07-31,57.894870,56.893623,56.893623
2019-08-31,57.980675,57.894870,114.788493
2019-09-30,62.527335,57.980675,115.875545
2019-10-31,67.823088,62.527335,120.508011
2019-11-30,65.232596,67.823088,130.350423


In [46]:
newp

array([[ 58.460857],
       [ 72.731094],
       [ 79.18988 ],
       [ 92.623436],
       [ 97.256516],
       [ 90.60903 ],
       [ 88.41096 ],
       [ 91.40355 ],
       [101.21677 ],
       [114.10296 ],
       [111.37168 ],
       [ 95.32473 ],
       [ 89.14867 ],
       [ 92.705734],
       [103.26439 ],
       [117.158035],
       [118.825806],
       [110.13256 ],
       [107.98466 ],
       [112.25867 ],
       [121.37558 ],
       [131.47118 ],
       [131.60414 ]], dtype=float32)

In [47]:
act_pred = pd.concat([pd.DataFrame(forecast_df.sarima_oilprice.values),pd.DataFrame(newp)], axis=1) 
act_pred.columns = ['Forecast_Oil_Price','Forecast_Iron_Price']

In [48]:
act_pred['date'] = forecast_df.index

In [49]:
act_pred = act_pred.set_index('date')
act_pred

,Forecast_Oil_Price,Forecast_Iron_Price
date,,
2019-07-31,57.894870,58.460857
2019-08-31,57.980675,72.731094
2019-09-30,62.527335,79.189880
2019-10-31,67.823088,92.623436
2019-11-30,65.232596,97.256516
2019-12-31,61.762357,90.609032
2020-01-31,64.043057,88.410957
2020-02-29,64.696372,91.403549
2020-03-31,71.018727,101.216766
